    conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

    <!-- https://github.com/pyg-team/pyg-lib -->

    conda install pyg -c pyg

    pip install -Uq sentence-transformers
    
    pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cu117.html

    pip install ogb

    conda install tqdm jupyter ipywidgets


In [16]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

1.13.1
11.7


In [2]:
from tqdm.notebook import tqdm_notebook

import os
import sys
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

# from torch_geometric.data import Data

from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer, util

In [3]:
if os.path.exists('LaBSE'):
    model = SentenceTransformer('LaBSE')
else:
    model = SentenceTransformer('sentence-transformers/LaBSE')
    model.save('LaBSE')

In [ ]:
import json

def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.
    
    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

In [8]:
queries_embeddings = model.encode(raw_query)
print(queries_embeddings.shape)

(3, 768)


---

  Torchtext

In [3]:
# !pip install -U torch==1.8.0 torchtext==0.9.0

# # Reload environment
# exit()

In [2]:
# from torchtext.legacy import data, datasetes

# filename = 'glove.6B.zip'
# url = 'http://nlp.stanford.edu/data/glove.6B.zip'
# torchtext.utils.download_from_url(url, filename)

# !unzip glove.6B.zip -d .vector_cache

# import torchtext
# from torchtext.legacy import data, datasetes

# def preprocess_queries(raw_query):
#     text = data.Field(sequential=True)
#     text.build_vocab([data.Example.fromlist([i], [("text", text)]) for i in raw_query], 
#                      vectors=torchtext.vocab.Vectors("glove.6B.50d.txt"))
#     queries = [text.process([i]) for i in raw_query]
#     queries = torch.cat(queries).squeeze(1)
#     return queries

# raw_query = ["search for laptops", "best camera under 500", "phone under 200"]
# queries = preprocess_queries(raw_query)

---

  Convert to Graph

In [12]:
# def convert_to_graph(raw_query, product_info, query_result, clicked_rank):
#     edge_index = []
#     x = []
#     edge_attr = []
#     query_index = {}
#     product_index = {}

#     for i, q in enumerate(raw_query):
#         query_index[q] = i
#         x.append(q)

#     for j, p in enumerate(product_info):
#         product_index[p[0]] = j + len(raw_query)
#         x.append(p)

#     for i, (q, result) in enumerate(query_result):
#         for r in result:
#             edge_index.append([query_index[q], product_index[r]])
#             edge_attr.append([clicked_rank[i][result.index(r)]])

#     edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
#     edge_attr = torch.tensor(edge_attr, dtype=torch.float)
#     x = torch.tensor(x, dtype=torch.float)

#     return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)


# product_info = [[1, "laptop", 500, 800, 650, 5, 10, 7],
#                 [2, "camera", 400, 450, 425, 3, 5, 4],
#                 [3, "phone", 150, 199, 175, 2, 3, 2.5]]
# query_result = [("search for laptops", [1]),
#                 ("best camera under 500", [2]),
#                 ("phone under 200", [3])]
# clicked_rank = [[1], [1], [1]]

# graph_data = convert_to_graph(queries_embeddings, product_info, query_result, clicked_rank)


In [11]:
# import pandas as pd

# # Load the query data into a pandas dataframe
# query_df = pd.DataFrame({
#     'raw_query': [
#         'لوستر سقفی برنز',
#         'قیمت هلیکوپتر',
#         'ساعت هوشمند'
#     ],
#     'result': [
#         [7151290, 6462477, 7385791, 8451497, None, 269...],
#         [363737, 3147253, 8720128, 9796388, 1420685, 5...],
#         [2459592, 9391819, 4229448, 7824893, 1670767, ...]
#     ],
#     'clicked_result': [
#         [9457219],
#         [3147253, 7135387],
#         [900897, 2931230, 31302]
#     ],
#     'clicked_rank': [
#         [16],
#         [1, 7],
#         [7, 25, 52]
#     ],
#     'timestamp': [
#         '2022-07-24T09:21:58.752000+00:00',
#         '2022-07-24T07:32:12.261000+00:00',
#         '2022-07-24T02:51:35.643000+00:00'
#     ]
# })

# # Load the product data into a pandas dataframe
# product_df = pd.DataFrame({
#     'id': [
#         1867826,
#         419611,
#         288575,
#         4614227,
#         4108604
#     ],
#     'category_name': [
#         'میکروسکوپ',
#         'ماشین اسباب بازی',
#         'کتاب و مجلات',
#         'لپ تاپ و نوت بوک',
#         'کتاب و مجلات'
#     ],
#     'titles': [
#         ['میکروسکوپ اپتیکی سلسترون مدل 44121 CGL'],
#         ['ماشین بازی فورد موستانگ مایستو مدل 2015 Ford ...'],
#         ['کتاب راهنمای کاربردی عیب یابی و تعمیر موتور س...'],
#         ['لپ تاپ مایکروسافت مدل سرفیس لپتاپ با پردازنده...'],
#         ['کتاب اقیانوسی

In [10]:
def convert_to_graph(raw_query, product_info, query_result, clicked_rank):
    edge_index = [] # edge index
    x = [] # node features
    edge_attr = [] # edge features
    query_index = {} # query index
    product_index = {} # product index

    # add query nodes
    for idx, query in enumerate(raw_query):
        query_index[idx] = idx
        x.append(query)

    # add product nodes
    for idx, p in enumerate(product_info):
        product_index[p[0]] = idx + len(raw_query)
        x.append(p)

    # add edges
    for idx, (q, result) in enumerate(query_result):
        for r in result:
            edge_index.append([query_index[q], product_index[r]])
            edge_attr.append([clicked_rank[idx][result.index(r)]])

    # convert to tensor
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    x = torch.tensor(x, dtype=torch.float)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

product_info = [[1, "laptop", 500, 800, 650, 5, 10, 7],
                [2, "camera", 400, 450, 425, 3, 5, 4],
                [3, "phone", 150, 199, 175, 2, 3, 2.5]]
query_result = [("search for laptops", [1]),
                ("best camera under 500", [2]),
                ("phone under 200", [3])]
clicked_rank = [[1], [1], [1]]

graph_data = convert_to_graph(queries_embeddings, product_info, query_result, clicked_rank)


NameError: name 'queries_embeddings' is not defined

In [8]:
# import pandas as pd
# import networkx as nx
# G = nx.DiGraph() # create a directed graph


In [ ]:
# To build a graph to learn to rank using PyTorch, you need to first preprocess your data to create the graph structure.
# One approach could be to treat each query-result pair as a node in the graph, and connect the nodes that belong to the same query. The graph can then be processed with graph neural networks (GNNs) that learn to propagate information between nodes.

# Here are the steps you can follow to build the graph:
# Encode the queries and results into numerical representations: You can use pre-trained models such as BERT or other embedding methods to encode the queries and results as dense vectors.
# Create nodes and edges in the graph: For each query-result pair, you can create a node and connect it to the other nodes that belong to the same query. You can use PyTorch's DGL library for building the graph structure.
# Train the graph neural network: You can use a GNN model such as Graph Attention Network (GAT) to learn the representation of each node and propagate information between nodes. The final representations of nodes can then be used for ranking.
# Evaluate the model: You can use standard evaluation metrics such as mean average precision (MAP) or normalized discounted cumulative gain (NDCG) to evaluate the performance of your model.
# This is a high-level overview of building a graph-based learn-to-rank model. Depending on your specific requirements, you may need to make adjustments to this process.

In [ ]:
from annoy import AnnoyIndex

# Create an Annoy index
index = AnnoyIndex(queries_embeddings.shape[1])

# Add the product embeddings to the index
for i, product_embedding in enumerate(product_embeddings):
    index.add_item(i, product_embedding)

# Build the index
index.build(10)  # 10 is the number of trees

# For each query, find the nearest neighbors (products)
for query_embedding in queries_embeddings:
    _, neighbors = index.get_nns_by_vector(query_embedding, n=10, search_k=-1, include_distances=True)
    # The `neighbors` list contains the indices of the nearest products to the query


----

In [ ]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

In [ ]:
print("Number of nodes:", data.x.shape[0])
print("Number of edges:", data.edge_index.shape[1])
print("Node features shape:", data.x.shape)
print("Edge indices shape:", data.edge_index.shape)

node_features = data.x
edge_indices = data.edge_index
print("Node features:", node_features)
print("Edge indices:", edge_indices)


In [ ]:
# Degree: The number of edges connected to a node. You can compute the degree of each node by summing the number of edges for each node.
import torch_geometric.utils as utils

degree = utils.degree(data.edge_index[0], num_nodes=data.num_nodes)
print("Degree of each node:", degree)


In [ ]:
# Adjacency matrix: A square matrix that shows the connections between nodes. You can compute the adjacency matrix by using the utils.to_sparse_adj function.

adj = utils.to_sparse_adj(data.edge_index, num_nodes=data.num_nodes)
print("Adjacency matrix:\n", adj)


In [ ]:
# Laplacian matrix: A matrix that captures the structure of a graph and is used in graph signal processing and graph convolutional networks. You can compute the Laplacian matrix by using the utils.normalized_cut function.

laplacian = utils.normalized_cut(data.edge_index, num_nodes=data.num_nodes)
print("Laplacian matrix:\n", laplacian)

In [ ]:
# Clustering coefficient: A measure of the degree to which nodes in a graph tend to cluster together. You can compute the clustering coefficient using the utils.cluster function.

cc = utils.cluster(data.edge_index, num_nodes=data.num_nodes)
print("Clustering coefficient of each node:", cc)

#### Graph-related properties

For a machine learning model for query relevance ranking, there are several graph-related features that you can explore to improve the performance of the model. Here are some of the most commonly used ones:

PageRank: A measure of the importance of each node in a graph, used to rank the relevance of web pages. You can compute the PageRank of each node by using the torch_geometric.nn.models.PageRank module.

HITS (Hyperlink-Induced Topic Search): A measure of the authority and hub-ness of each node in a graph, used to rank the relevance of web pages. You can compute the HITS scores of each node by using the torch_geometric.nn.models.Hits module.

Betweenness centrality: A measure of the number of times a node lies on the shortest path between two other nodes, used to rank the importance of nodes in a graph. You can compute the betweenness centrality of each node by using the torch_geometric.nn.models.BetweennessCentrality module.

Closeness centrality: A measure of the average distance between a node and all other nodes in a graph, used to rank the proximity of nodes in a graph. You can compute the closeness centrality of each node by using the torch_geometric.nn.models.ClosenessCentrality module.

Eigenvector centrality: A measure of the influence of a node in a graph, used to rank the importance of nodes in a graph. You can compute the eigenvector centrality of each node by using the torch_geometric.nn.models.EigenvectorCentrality module.

Local clustering coefficient: A measure of the number of triangles (sets of three nodes that are all connected) that are connected to a node, used to rank the importance of nodes in a graph. You can compute the local clustering coefficient of each node by using the torch_geometric.nn.models.LocalClusteringCoefficient module.

Personalized PageRank: A variant of PageRank that allows for personalized rankings based on the query and user interests. You can compute the personalized PageRank of each node by using the torch_geometric.nn.models.PersonalizedPageRank module.

These are just a few examples of graph-related features that you can use to improve the performance of a query relevance ranking model. There may be other features that are relevant for your specific use case, so it is important to explore and evaluate different options to see what works best for your problem.

---

Relevance scores: The relevance scores indicate how relevant a product is to a query. These scores can be used as the target values for the ranking model.
Query-product pairs: The query-product pairs represent the interactions between queries and products. These pairs can be used as the input data for the ranking model.
Features: Features can be used to represent the characteristics of the products and queries. These features can include information about the product, the query, or the interaction between the two.
Evaluation metrics: Evaluation metrics can be used to measure the performance of the ranking model. Some commonly used metrics for ranking models include precision, recall, mean average precision (MAP), normalized discounted cumulative gain (NDCG), and area under the ROC curve (AUC).
Hyperparameters: The ranking model may have several hyperparameters that can be tuned to improve its performance. These hyperparameters can include the learning rate, the batch size, the number of epochs, the number of hidden layers, and the number of nodes in each layer.
Cross-validation: Cross-validation is a technique for evaluating the performance of a model by dividing the data into training and validation sets. The model is trained on the training set and evaluated on the validation set. This allows you to measure the model's performance on unseen data and prevent overfitting.
Ensemble methods: Ensemble methods can be used to combine the predictions of multiple models to improve the performance of the ranking model. Some commonly used ensemble methods include bagging, boosting, and stacking.

These are just a few examples of properties and metrics that you can explore when working with a learning to rank problem as query-product relevancy ranking. The specific properties and metrics that you should explore will depend on your specific use case and the data that you have available.

Mean Average Precision (MAP): A measure of the average precision of a ranked list of documents, where precision is the number of relevant documents retrieved divided by the number of retrieved documents.

Normalized Discounted Cumulative Gain (NDCG): A measure of the quality of the ranking of documents, taking into account both the relevance of the documents and their position in the ranking.

Precision@K: The precision of a ranked list of documents, computed as the number of relevant documents in the top K positions divided by K.

Recall@K: The recall of a ranked list of documents, computed as the number of relevant documents in the top K positions divided by the total number of relevant documents.

F1-Score@K: The harmonic mean of precision and recall at position K.

Mean Reciprocal Rank (MRR): The average reciprocal rank of the first relevant document in a ranked list of documents.

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data

from sentence_transformers import SentenceTransformer

In [4]:
sentences = ["The cat is sitting on the mat.",
             "A dog is barking outside.",
             "The sun is shining brightly.",
             "A bird is singing a beautiful song.",
             "The wind is blowing gently.",
             "A flower is blooming in the garden.",
             "A car is driving down the street.",
             "The rain is falling from the sky.",
             "A tree is growing tall and strong.",
             "A person is walking down the sidewalk.",
             "A plane is flying overhead.",
             "The ocean is vast and endless.",
             "A mountain is towering above the landscape.",
             "The sky is a brilliant shade of blue.",
             "A river is flowing peacefully.",
             "A butterfly is flitting from flower to flower.",
             "The stars are twinkling in the night sky.",
             "A deer is running through the forest.",
             "A city is bustling with activity.",
             "The moon is shining brightly.",]

In [ ]:
model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings = model.encode(sentences)


In [ ]:
# # Step 1: Embed each sentence into a fixed-length vector using a sentence embedding model
# def get_sentence_embedding(sentence, model):
#     sentence_embedding = model.encode([sentence])
#     return sentence_embedding

# sentences = [...]  # Your set of sentences
num_sentences = len(sentences)
embedding_dim = 768  # The dimension of the sentence embedding

sentence_embeddings = np.zeros((num_sentences, embedding_dim))
for i in range(num_sentences):
    sentence_embeddings[i, :] = get_sentence_embedding(sentences[i], model)

# Step 2: Compute the similarity between each pair of sentences using cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

similarity_matrix = np.zeros((num_sentences, num_sentences))
for i in range(num_sentences):
    for j in range(i + 1, num_sentences):
        similarity_matrix[i, j] = cosine_similarity(sentence_embeddings[i, :], sentence_embeddings[j, :])
        similarity_matrix[j, i] = similarity_matrix[i, j]

# Step 3: Connect nodes that are similar to each other with edges
threshold = 0.7 # A similarity threshold that determines whether two sentences are considered similar

edge_index = []
for i in range(num_sentences):
    for j in range(i + 1, num_sentences):
        if similarity_matrix[i, j] >= threshold:
            edge_index.append([i, j])

edge_index = np.array(edge_index)

# Create PyTorch Geometric data object
x = torch.tensor(sentence_embeddings, dtype=torch.float)
data = Data(x=x, edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous())


In [ ]:
# show values
print("node values:", data.x)
print("edge indices:", data.edge_index)

In [ ]:
# Describe the graph
print("Number of nodes:", data.x.shape[0])
print("Number of edges:", data.edge_index.shape[1])
print("Node features shape:", data.x.shape)
print("Edge indices shape:", data.edge_index.shape)

# show the graph
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
G.add_nodes_from(range(data.x.shape[0]))
G.add_edges_from(data.edge_index.t().numpy())
nx.draw(G, with_labels=True)
plt.show()


In [ ]:
centrality = nx.eigenvector_centrality(G)
print("Eigenvector centrality of each node:", centrality)

betweenness = nx.betweenness_centrality(G)
print("Betweenness centrality of each node:", betweenness)


In [ ]:
import json

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from torch_geometric.data import Data

from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/LaBSE')
# # Step 1: Embed each sentence into a fixed-length vector using a sentence embedding model
def get_sentence_embedding(sentence, model):
    sentence_embedding = model.encode([sentence])
    return sentence_embedding

In [9]:
import json
NUM_TRAIN_SAMPLES = None

def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.
    
    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

In [10]:
query = pd.DataFrame(read_json_lines('./Torob/torob-search-data_v1.jsonl'))

In [11]:
query.columns # Index(['raw_query', 'result', 'clicked_result', 'clicked_rank', 'timestamp'], dtype='object')

query.head()

,raw_query,result,clicked_result,clicked_rank,timestamp
0,لوستر سقفی برنز,"[7151290, 6462477, 7385791, 8451497, None, 269...",[9457219],[16],2022-07-24T09:21:58.752000+00:00
1,قیمت هلیکوپتر,"[363737, 3147253, 8720128, 9796388, 1420685, 5...","[3147253, 7135387]","[1, 7]",2022-07-24T07:32:12.261000+00:00
2,ساعت هوشمند,"[2459592, 9391819, 4229448, 7824893, 1670767, ...","[900897, 2931230, 31302]","[7, 25, 52]",2022-07-24T02:51:35.643000+00:00
3,تفلون مایع,"[5428407, 4474271, 4581189, 8504749, 4131340, ...","[7660686, 5901997, 2376830, 2383125]","[19, 18, 21, 36]",2022-07-24T02:51:54.771000+00:00
4,خط زن,"[8466568, 9285461, 1105044, 1314054, 1791930, ...",[3852139],[20],2022-07-23T20:56:36.043000+00:00


In [ ]:
# each query is a node in the graph and each result is another node in the graph
# each edge is a connection between a query and a result (if the result is clicked) or between two results (if they are similar)

x = []
edge_index = []
for i, row in query.iterrows():
    print(row)
    x.append(get_sentence_embedding(row['raw_query'], model))
        
x = torch.from_numpy(np.array(x))

# edge_index = np.array(edge_index)

# data = Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous())

In [ ]:
x_text = torch.tensor(text_node_features, dtype=torch.float)
x_user = torch.tensor(user_node_features, dtype=torch.float)
data = Data(x_text=x_text, x_user=x_user, edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous())


In [ ]:
num_sentences = len(sentences)
embedding_dim = 768  # The dimension of the sentence embedding

sentence_embeddings = np.zeros((num_sentences, embedding_dim))
for i in range(num_sentences):
    sentence_embeddings[i, :] = get_sentence_embedding(sentences[i], model)

In [ ]:
x = torch.tensor(sentence_embeddings, dtype=torch.float)
